In [1]:
import os
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.lines as mlines
from IPython.display import Video, Image
import imageio
import pdb
from fastdtw import fastdtw
from scipy.spatial.distance import euclidean

In [ ]:
#ONLY DATA TO BE PROVIDED BY YOU
data_path = '/path/to/inference/data' #(Usally in outdir/inference/filtered_resampled_inverted_both)
test_number = 2450

In [ ]:
#Original data filepaths 
#Can be downloaded from CSF23 Data Release
true_data_path = "AVTacotron2_data/CSF23/"
true_lips_path = true_data_path + "MP_lips/par"+str(test_number)+".npy"
true_hands_path = true_data_path + "MP_hands/par"+str(test_number)+".npy"

In [ ]:
text_file = 'filelists/.../test_cslm2023.txt'  
# Read the file and search for the line
with open(text_file, 'r') as file:
    for line in file:
        if line.startswith(f"par{test_number}|"):
            # Extract the part of the line after the "|"
            part_after_pipe = line.split("|")[1].strip()
            print(f"The sentence is {part_after_pipe}")
            break  # Stop after finding the first occurrence

The sentence is §Le délai de quinze jours est donc raisonnable.§


In [ ]:
#Predicted data filepaths
lips_file_path = data_path+'par'+f"{test_number:04d}"+'_lips.npy'
hands_file_path = data_path+'par'+f"{test_number:04d}"+'_hands.npy'
output_path = os.path.dirname(data_path) + "/"

In [6]:
# Determine output folder based on the npy file name
base_name = os.path.basename(lips_file_path)  # Gets the file name from the path
output_name = os.path.splitext(base_name)[0]  # Removes the file extension
output_folder = output_path + output_name + "_dtw"

In [7]:
true_data = np.load(true_lips_path)
predicted_data = np.load(lips_file_path)

# Function to apply DTW for multidimensional data using a custom Euclidean distance
def apply_multidimensional_fastdtw(orig_data, pred_data):
    # Use Euclidean distance, as scipy's euclidean function can handle multidimensional vectors
    distance, path = fastdtw(orig_data, pred_data, dist=euclidean)
    return distance, path

# Get DTW distance and the optimal path
distance, path = apply_multidimensional_fastdtw(true_data, predicted_data)

# Retrieve the aligned series based on the DTW path
aligned_predicted_data = np.array([predicted_data[j] for _, j in path])

In [8]:
def plot_and_save_lips(data, true_data, lips_file_path):
   
    # Check if the directory exists, and if not, create it
    if not os.path.exists(output_folder): 
        os.makedirs(output_folder)
    
    data = 1 - data #Because MP coordinate origin is top right
    true_data = 1 - true_data #Because MP coordinate origin is top right
    num_points = 42  # Number of x and y coordinates
    for i, row in enumerate(data):
        x = row[:num_points]
        y = row[num_points:num_points*2]
#         pdb.set_trace()
        
        if i<true_data.shape[0]:
            x_true = true_data[i-1,:num_points]
            y_true = true_data[i-1,num_points:num_points*2]
        
        plt.figure(figsize=(10, 6))

        # Scatter plot of all points
        plt.scatter(x[:20], y[:20], c='green')  # First circle points
        plt.scatter(x[20:], y[20:], c='green')  # Second circle points
        plt.scatter(x_true[:20], y_true[:20], c='red')  # First circle points
        plt.scatter(x_true[20:], y_true[20:], c='red')  # Second circle points

        # Connect the first 20 points in a circle
        for j in range(20):
            next_point = (j + 1) % 20  # Ensures the last point connects to the first
            plt.plot([x[j], x[next_point]], [y[j], y[next_point]], 'g-')
            plt.plot([x_true[j], x_true[next_point]], [y_true[j], y_true[next_point]], 'r-')

        # Connect the next 21 points in a circle
        for j in range(21, 41):
            if j == 40:  # The last point should connect to the first point of this group
                next_point = 21
            else:
                next_point = j + 1
            plt.plot([x[j], x[next_point]], [y[j], y[next_point]], 'g-')
            plt.plot([x_true[j], x_true[next_point]], [y_true[j], y_true[next_point]], 'r-')
            

#         plt.title(f"Frame {i}")
        plt.xlabel("X Coordinates", fontsize=20)
        plt.ylabel("Y Coordinates", fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)
        plt.xlim([min(np.min(data[:, :num_points]),np.min(true_data[:, :num_points])), max(np.max(data[:, :num_points]),np.max(true_data[:, :num_points]))])
        plt.ylim([min(np.min(data[:, num_points:num_points*2]), np.min(true_data[:, num_points:num_points*2])), max(np.max(data[:, num_points:num_points*2]), np.max(true_data[:, num_points:num_points*2]))])
        
        # Create custom legend handles
        predicted_line = mlines.Line2D([], [], color='green', label='Predicted')
        expected_line = mlines.Line2D([], [], color='red', label='Expected')

        # Add the legend to the plot
        plt.legend(handles=[predicted_line, expected_line], loc = 'upper right', fontsize=20)

        plt.savefig(f"{output_folder}/frame_{i:04d}.png")
        plt.close()

# Example usage with the path to the .npy file
plot_and_save_lips(aligned_predicted_data, true_data, lips_file_path)

In [9]:
def create_video(image_folder, output_video_file):
    images = []
    for i in range(len(aligned_predicted_data)):  # assuming one image per row in data
        images.append(imageio.imread(f"{image_folder}/frame_{i:04d}.png"))
    imageio.mimsave(output_video_file, images, fps=15)

create_video(output_folder, output_folder+"/"+output_name+'_15fps.mp4')

/tmp/ipykernel_1000446/4136327886.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"{image_folder}/frame_{i:04d}.png"))
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 600) to (1008, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [10]:
display(Video(output_folder+"/"+output_name+"_15fps.mp4", embed=True))

In [11]:
# Determine output folder based on the npy file name
base_name = os.path.basename(hands_file_path)  # Gets the file name from the path
output_name = os.path.splitext(base_name)[0]  # Removes the file extension
output_folder = output_path + output_name + "_dtw"

In [12]:
true_data = np.load(true_hands_path)
predicted_data = np.load(hands_file_path)

# Get DTW distance and the optimal path
distance, path = apply_multidimensional_fastdtw(true_data, predicted_data)

# Retrieve the aligned series based on the DTW path
aligned_predicted_data = np.array([predicted_data[j] for _, j in path])

In [ ]:
import pdb
def plot_and_save_hands(data, true_data, hands_file_path):
   
    # Check if the directory exists, and if not, create it
    if not os.path.exists(output_folder):
        os.makedirs(output_folder)
        
    num_points = 21  # Number of x and y coordinates
    
    # Define the connections between points
    connections = [
        (0, 1), (1, 2), (2, 3), (3, 4),  # Thumb
        (0, 5), (5, 6), (6, 7), (7, 8),  # Index finger
        (0, 9), (9, 10), (10, 11), (11, 12),  # Middle finger
        (0, 13), (13, 14), (14, 15), (15, 16),  # Ring finger
        (0, 17), (17, 18), (18, 19), (19, 20),  # Pinky finger
    ]
        

    data = 1 - data #Because MP coordinate origin is top right
    true_data = 1 - true_data #Because MP coordinate origin is top right

    for i, row in enumerate(data):
        x = row[:num_points]
        y = row[num_points:num_points*2]
        
        if i<true_data.shape[0]:
            x_true = true_data[i-1,:num_points]
            y_true = true_data[i-1,num_points:num_points*2]
        
        # Calculating the mean
        mean_x = np.mean(x)
        mean_y = np.mean(y)

        # Conditionally modifying x and y if it's "no-shape/no-hand" position
        if mean_x > 0.8 and mean_y > 0.8:
            x = np.ones_like(x)
            y = np.ones_like(y)
        
        plt.figure(figsize=(10, 6))
        # Plot the points
        plt.scatter(x, y, c='green')
        plt.scatter(x_true, y_true, c='red')
        
        # Draw the connections
        for start, end in connections:
            plt.plot([x[start], x[end]], [y[start], y[end]], c='green')
            plt.plot([x_true[start], x_true[end]], [y_true[start], y_true[end]], c='red')


        plt.xlabel("X Coordinates", fontsize=20)
        plt.ylabel("Y Coordinates", fontsize=20)
        plt.xticks(fontsize=20)
        plt.yticks(fontsize=20)

        plt.xlim(0.2,0.6)
        plt.ylim(0,0.8)

        # Create custom legend handles
        predicted_line = mlines.Line2D([], [], color='green', label='Predicted')
        expected_line = mlines.Line2D([], [], color='red', label='Expected')

        # Add the legend to the plot
        plt.legend(handles=[predicted_line, expected_line], loc = 'upper right', fontsize=20)
        
        plt.savefig(f"{output_folder}/frame_{i:04d}.png")
        plt.close()

# Example usage with the path to the .npy file
plot_and_save_hands(aligned_predicted_data, true_data, hands_file_path)

In [14]:
def create_video(image_folder, output_video_file):
    images = []
    for i in range(len(aligned_predicted_data)):  # assuming one image per row in data
        images.append(imageio.imread(f"{image_folder}/frame_{i:04d}.png"))
    imageio.mimsave(output_video_file, images, fps=15)

create_video(output_folder, output_folder+"/"+output_name+'_15fps.mp4')

/tmp/ipykernel_1000446/4136327886.py:4: DeprecationWarning: Starting with ImageIO v3 the behavior of this function will switch to that of iio.v3.imread. To keep the current behavior (and make this warning disappear) use `import imageio.v2 as imageio` or call `imageio.v2.imread` directly.
  images.append(imageio.imread(f"{image_folder}/frame_{i:04d}.png"))
IMAGEIO FFMPEG_WRITER WARNING: input image is not divisible by macro_block_size=16, resizing from (1000, 600) to (1008, 608) to ensure video compatibility with most codecs and players. To prevent resizing, make your input image divisible by the macro_block_size or set the macro_block_size to 1 (risking incompatibility).


In [15]:
display(Video(output_folder+"/"+output_name+"_15fps.mp4", embed=True))